# Final Project Analysis




<font size="5"> Introduction </font> 

In [2]:
### Run this cell before continuing. 
library(tidyverse)
library(repr)
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [4]:
game_data <- read_csv ("data/players.csv")
game_data

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,17
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


<font size="3"> Activity #1: Explore Datasets - Preliminary </font> 

Using what you have learnt in weeks 1-4, read the dataset, take a look at it, and write a short description about the dataset.

Some questions you should try to answer:
What is the dataset about?
How many variables are there?
How many observations are there?

Data Description: identify and fully describe the dataset that was used to answer the question. Provide a full descriptive summary of the dataset, including information such as the number of observations, summary statistics, number of variables, name and type of variables, what the variables mean, any issues you see in the data, any other potential issues related to things you cannot directly see, how the data were collected, etc. Make sure to use bullet point lists or tables to summarize the variables in an easy-to-understand format. Note that the selected dataset(s) will probably contain more variables than you need. 


*****delete the above later****

**Question: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?**

The dataset explores a Minecraft server dataset from a UBC research group led by Frank Woods. [...]



There are 7 variables, with names as follows:

`experience` (character): the player's skill level (listed as either amateur, beginner, regular, pro, or veteran)

`subscribe` (logical): if the player has subscribed to the game-related newsletter

`hashedEmail`(character): the player's unique identifier  

`played_hours`(double): how many hours each player has played the game

`names`(character): name of each player

`gender`(character): gender of each player

`Age`(double): age of each player


Based on the game_data above, there are 196 rows corresponding to each player, meaning that there are 196 observations. 




<font size="5"> Methods </font> 

Try to answer these questions now:
Identify the main outcome/categorical/label variable in the dataset.

How many values/groups are in this variable?
How many observations are there in each value/group?

*****delete the above later****



As the main question for this dataset is to see the player characteristics and behaviours that are most predictive of whether or not they have subscribed to the game-related newsletter, then `subscribe` would be the main categorical label. This logical variable has only two possibilities: `TRUE` (that they have subscribed) or `FALSE` (that they are not subscribed to the newsletter).




<font size="5"> Results </font> 

Make some visualizations of the outcome variable:

What does the distribution of the variable look like?
What relationship does it have with some of the other variables?
Tip: Try using a range of box plots, scatterplots, bar charts, line graphs, etc.

In [14]:
#testing
game_plot <- game_data |>
    ggplot (aes (x = subscribe, y = Age)) +
    geom_point (alpha = 0.5) +
    labs (x = "Subscribed to Newsletter", y = "") + 
    theme (text = element_text (size = 20))

#game_plot


<font size="5"> Discussion </font> 

- summarize what you found
- discuss whether this is what you expected to find?
- discuss what impact could such findings have?
- discuss what future questions could this lead to?


todo: 
- add background information
- standardize data
- figure out which k-nearest neighbours is best for this classification problem
- tuning, splitting into training + testing data
- finding accuracy/precision/recall
- discussion

In [6]:
# source("cleanup.R")??